In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv1D, Activation, SpatialDropout1D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf


df = pd.read_excel('Philadelphia.xlsx')
df.dropna(inplace=True)


df['Hour'] = pd.to_datetime(df['Time']).dt.hour
df['DayOfWeek'] = pd.to_datetime(df['Time']).dt.dayofweek
df['Month'] = pd.to_datetime(df['Time']).dt.month
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)


for lag in range(1, 25):
    df[f'Lag_{lag}'] = df['EnergyDemand'].shift(lag)


df['Temp_RollingMean_24'] = df['Temperature'].rolling(window=24).mean()
df['Temp_RollingStd_24'] = df['Temperature'].rolling(window=24).std()


df['Temp_Humidity_Interaction'] = df['Temperature'] * df['RelativeHumidity']


df.dropna(inplace=True)

features = df[['Temperature', 'DewPoint', 'RelativeHumidity', 'Precipitation', 'WindSpeed', 'Pressure',
               'Hour', 'DayOfWeek', 'Month', 'IsWeekend',
               'Lag_1', 'Lag_2', 'Lag_3', 'Lag_4', 'Lag_5', 'Lag_6', 'Lag_7', 'Lag_8', 'Lag_9', 'Lag_10',
               'Lag_11', 'Lag_12', 'Lag_13', 'Lag_14', 'Lag_15', 'Lag_16', 'Lag_17', 'Lag_18', 'Lag_19',
               'Lag_20', 'Lag_21', 'Lag_22', 'Lag_23', 'Lag_24',
               'Temp_RollingMean_24', 'Temp_RollingStd_24',
               'Temp_Humidity_Interaction']].values
energy_demand = df['EnergyDemand'].values.reshape(-1, 1)
dates = df['Time'].values


scaler_features = MinMaxScaler()
scaler_energy = MinMaxScaler()
features_normalized = scaler_features.fit_transform(features)
energy_demand_normalized = scaler_energy.fit_transform(energy_demand)

def create_sequences(data, target, dates, seq_length):
    sequences = []
    targets = []
    sequence_dates = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_seq = target[i+seq_length]
        date_seq = dates[i+seq_length]
        sequences.append(seq)
        targets.append(target_seq)
        sequence_dates.append(date_seq)
    return np.array(sequences), np.array(targets), np.array(sequence_dates)

seq_length = 24
X, y, dates_seq = create_sequences(features_normalized, energy_demand_normalized, dates, seq_length)

def residual_block(x, dilation_rate, nb_filters, kernel_size, dropout_rate=0.2):
    prev_x = x
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)


    if prev_x.shape[-1] != x.shape[-1]:
        prev_x = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(prev_x)

    x = tf.keras.layers.add([prev_x, x])
    return x

def create_model(input_shape, nb_filters, kernel_size, dilations, nb_stacks):
    input_layer = Input(shape=input_shape)
    x = input_layer
    for _ in range(nb_stacks):
        for dilation_rate in dilations:
            x = residual_block(x, dilation_rate, nb_filters, kernel_size)
    x = Lambda(lambda tt: tt[:, -1, :])(x)
    output_layer = Dense(1)(x)
    model = tf.keras.models.Model(input_layer, output_layer)
    return model

kf = KFold(n_splits=5, shuffle=False, random_state=None)
mse_scores = []
mae_scores = []
mape_scores = []

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

result_df = pd.DataFrame()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dates_test = dates_seq[test_index]

    model = create_model(
        input_shape=(seq_length, X.shape[2]),
        nb_filters=64,
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32],
        nb_stacks=1
    )

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)


    predictions_normalized = model.predict(X_test)
    predictions = scaler_energy.inverse_transform(predictions_normalized)
    y_test_original = scaler_energy.inverse_transform(y_test)


    mse = mean_squared_error(y_test_original, predictions)
    mae = mean_absolute_error(y_test_original, predictions)
    mape = mean_absolute_percentage_error(y_test_original, predictions)

    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)

    fold_results = pd.DataFrame({
        'Date': dates_test,
        'Actual': y_test_original.flatten(),
        'Predicted': predictions.flatten()
    })
    result_df = pd.concat([result_df, fold_results], ignore_index=True)


average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_mape = np.mean(mape_scores)
print(f"Average Mean Squared Error: {average_mse}")
print(f"Average Mean Absolute Error: {average_mae}")
print(f"Average Mean Absolute Percentage Error: {average_mape}")


plt.figure(figsize=(12, 6))
plt.plot(result_df['Actual'][:100], label='Actual')
plt.plot(result_df['Predicted'][:100], label='Predicted')
plt.title('Predicted vs Actual Energy Demand')
plt.xlabel('Time')
plt.ylabel('Energy Demand')
plt.legend()
plt.show()

ADF Statistic: -13.876101
p-value: 0.000000
Critical Values:
	1%: -3.430
	5%: -2.862
	10%: -2.567
ADF Statistic: -35.930386
p-value: 0.000000
Critical Values:
	1%: -3.430
	5%: -2.862
	10%: -2.567
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=9.60 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=122644.753, Time=0.29 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=117680.151, Time=6.01 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=10.12 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=122634.585, Time=0.29 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=117473.955, Time=21.91 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=64.58 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=20.45 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=117544.169, Time=4.58 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=117394.604, Time=15.71 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=117567.586, Time=8.71 sec
 ARIMA(2,1,0)(2,1,1)[12] 

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


NaN values found in forecast or test data.
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=inf, Time=16.19 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=316711.403, Time=0.45 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=302520.861, Time=29.62 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=49.05 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=307392.981, Time=0.94 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=300663.717, Time=105.32 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=164.48 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=45.47 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=310040.475, Time=19.76 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=296416.275, Time=111.82 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=298153.828, Time=39.00 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=172.95 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=inf, Time=61.47 sec
 ARIMA(3,1,0)(2,1,0)[12]             : AIC=2

: 